In [2]:
#code to read data from snowflake using FosforML
 
from fosforml.model_manager.snowflakesession import get_session

my_session = get_session()
 
table_name = 'MONTH_DATA_202408'
 
sf_df = my_session.sql("select * from {}".format(table_name)) 

df = sf_df.to_pandas() 
 
print(df.shape) 

df.head(100) 

(74960, 18)


,MNTH_CODE,TRANS_DATE,START_DATE,SALES_VALUE,SALES_UNITS,SALES_VOLUME,SALES_PTR_VALUE,OC_CODE,DISTRIBUTOR_CODE,OUTLET_CODE,CITY,STATE,COUNTY,STREET,PRODUCT_CODE,CATEGORY,SUBCATEGORY,BRAND
0,202408,2024-08-03,2024-07-31,2610.18,16,0.004560,2690.909091,202208,DB0110,OL33833,Garden Grove,California,City Center,Str1,PRD0123,Kids Care,Baby Shampoo,Ruby
1,202408,2024-08-11,2024-07-31,245.45,2,0.000500,245.454545,202208,DB0110,OL65445,Albany,Georgia,Dolphin,Str3,PRD0107,Lotion,Body Lotion,Coral
2,202408,2024-08-17,2024-07-31,277.14,32,0.000768,285.714286,202208,DB0110,OL127954,Johnstown,New York,Dolphin,Str2,PRD0106,Hair Care,Shampoo,Hair & Shoulders
3,202408,2024-08-11,2024-07-31,345.45,2,0.000750,345.454545,202208,DB0706,OL12634,New Kensington,Pennsylvania,City Center,Str5,PRD0113,Perfume and Deodrants,Unisex Deodrant,Navy Blue
4,202408,2024-08-13,2024-07-31,138.57,16,0.000400,142.857143,202208,DB0110,OL97127,Concord,Massachusetts,Dolphin,Str4,PRD0147,Kids Care,Baby Cream,Mint
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,202408,2024-08-09,2024-07-31,672.73,4,0.001200,672.727273,202208,DB0706,OL81758,Sebring,Florida,City Center,Str5,PRD0117,Soap,Medicated Soap,Dark green
96,202408,2024-08-26,2024-07-31,1059.09,6,0.002250,1059.090909,202208,DB0652,OL81680,Gila Bend,Arizona,City Center,Str3,PRD0113,Perfume and Deodrants,Unisex Deodrant,Navy Blue
97,202408,2024-08-12,2024-07-31,128.18,3,0.000600,128.181818,202208,DB0209,OL81260,Revere,Massachusetts,City Center,Str5,PRD0038,Dental,ToothPaste,Toothy Kids
98,202408,2024-08-16,2024-07-31,107.14,6,0.000390,107.142857,202208,DB0209,OL81405,Scranton,Pennsylvania,City Center,Str5,PRD0009,Dental,ToothPaste,Toothy Coal


In [3]:
type(sf_df)

snowflake.snowpark.dataframe.DataFrame

In [4]:
type(df)

pandas.core.frame.DataFrame

In [5]:
df.describe()

,MNTH_CODE,SALES_VALUE,SALES_UNITS,SALES_VOLUME,SALES_PTR_VALUE,OC_CODE
count,74960.0,74960.000000,74960.000000,74960.000000,74960.000000,74960.0
mean,202408.0,448.120112,12.311286,0.000923,459.903061,202208.0
std,0.0,861.985547,42.341019,0.001701,909.228371,0.0
min,202408.0,4.460000,1.000000,0.000011,4.464286,202208.0
25%,202408.0,137.140000,3.000000,0.000384,142.857143,202208.0
50%,202408.0,181.820000,6.000000,0.000448,181.818182,202208.0
75%,202408.0,488.180000,16.000000,0.000900,488.181818,202208.0
max,202408.0,42630.000000,6000.000000,0.092950,43500.000000,202208.0


In [6]:
df.corr(numeric_only=True)

,MNTH_CODE,SALES_VALUE,SALES_UNITS,SALES_VOLUME,SALES_PTR_VALUE,OC_CODE
MNTH_CODE,NaN,NaN,NaN,NaN,NaN,NaN
SALES_VALUE,NaN,1.000000,0.314582,0.927095,0.997685,NaN
SALES_UNITS,NaN,0.314582,1.000000,0.421160,0.322620,NaN
SALES_VOLUME,NaN,0.927095,0.421160,1.000000,0.923330,NaN
SALES_PTR_VALUE,NaN,0.997685,0.322620,0.923330,1.000000,NaN
OC_CODE,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74960 entries, 0 to 74959
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   MNTH_CODE         74960 non-null  int32  
 1   TRANS_DATE        74960 non-null  object 
 2   START_DATE        74960 non-null  object 
 3   SALES_VALUE       74960 non-null  float64
 4   SALES_UNITS       74960 non-null  int16  
 5   SALES_VOLUME      74960 non-null  float64
 6   SALES_PTR_VALUE   74960 non-null  float64
 7   OC_CODE           74960 non-null  int32  
 8   DISTRIBUTOR_CODE  74960 non-null  object 
 9   OUTLET_CODE       74960 non-null  object 
 10  CITY              74960 non-null  object 
 11  STATE             74960 non-null  object 
 12  COUNTY            74960 non-null  object 
 13  STREET            74960 non-null  object 
 14  PRODUCT_CODE      74960 non-null  object 
 15  CATEGORY          74960 non-null  object 
 16  SUBCATEGORY       74960 non-null  object

In [8]:
df.columns

Index(['MNTH_CODE', 'TRANS_DATE', 'START_DATE', 'SALES_VALUE', 'SALES_UNITS',
       'SALES_VOLUME', 'SALES_PTR_VALUE', 'OC_CODE', 'DISTRIBUTOR_CODE',
       'OUTLET_CODE', 'CITY', 'STATE', 'COUNTY', 'STREET', 'PRODUCT_CODE',
       'CATEGORY', 'SUBCATEGORY', 'BRAND'],
      dtype='object')

In [9]:
df.isnull().sum()

MNTH_CODE           0
TRANS_DATE          0
START_DATE          0
SALES_VALUE         0
SALES_UNITS         0
SALES_VOLUME        0
SALES_PTR_VALUE     0
OC_CODE             0
DISTRIBUTOR_CODE    0
OUTLET_CODE         0
CITY                0
STATE               0
COUNTY              0
STREET              0
PRODUCT_CODE        0
CATEGORY            0
SUBCATEGORY         0
BRAND               0
dtype: int64

In [10]:
df.duplicated().sum()

8

In [11]:
from sklearn.preprocessing import OneHotEncoder

In [12]:
data=df.copy()

In [13]:
data[['MNTH_CODE', 'TRANS_DATE', 'START_DATE', 'SALES_VALUE', 'SALES_UNITS',
       'SALES_VOLUME', 'SALES_PTR_VALUE', 'OC_CODE', 'DISTRIBUTOR_CODE',
       'OUTLET_CODE', 'CITY', 'STATE', 'COUNTY', 'STREET', 'PRODUCT_CODE',
       'CATEGORY', 'SUBCATEGORY', 'BRAND']].head()

,MNTH_CODE,TRANS_DATE,START_DATE,SALES_VALUE,SALES_UNITS,SALES_VOLUME,SALES_PTR_VALUE,OC_CODE,DISTRIBUTOR_CODE,OUTLET_CODE,CITY,STATE,COUNTY,STREET,PRODUCT_CODE,CATEGORY,SUBCATEGORY,BRAND
0,202408,2024-08-03,2024-07-31,2610.18,16,0.004560,2690.909091,202208,DB0110,OL33833,Garden Grove,California,City Center,Str1,PRD0123,Kids Care,Baby Shampoo,Ruby
1,202408,2024-08-11,2024-07-31,245.45,2,0.000500,245.454545,202208,DB0110,OL65445,Albany,Georgia,Dolphin,Str3,PRD0107,Lotion,Body Lotion,Coral
2,202408,2024-08-17,2024-07-31,277.14,32,0.000768,285.714286,202208,DB0110,OL127954,Johnstown,New York,Dolphin,Str2,PRD0106,Hair Care,Shampoo,Hair & Shoulders
3,202408,2024-08-11,2024-07-31,345.45,2,0.000750,345.454545,202208,DB0706,OL12634,New Kensington,Pennsylvania,City Center,Str5,PRD0113,Perfume and Deodrants,Unisex Deodrant,Navy Blue
4,202408,2024-08-13,2024-07-31,138.57,16,0.000400,142.857143,202208,DB0110,OL97127,Concord,Massachusetts,Dolphin,Str4,PRD0147,Kids Care,Baby Cream,Mint


In [14]:
import pandas as pd

In [15]:
status=pd.get_dummies(data[['MNTH_CODE', 'TRANS_DATE', 'START_DATE', 'SALES_VALUE', 'SALES_UNITS',
       'SALES_VOLUME', 'SALES_PTR_VALUE', 'OC_CODE', 'DISTRIBUTOR_CODE',
       'OUTLET_CODE', 'CITY', 'STATE', 'COUNTY', 'STREET', 'PRODUCT_CODE',
       'CATEGORY', 'SUBCATEGORY', 'BRAND']])

In [16]:
status.head()

,MNTH_CODE,SALES_VALUE,SALES_UNITS,SALES_VOLUME,SALES_PTR_VALUE,OC_CODE,TRANS_DATE_2024-08-01,TRANS_DATE_2024-08-02,TRANS_DATE_2024-08-03,TRANS_DATE_2024-08-04,...,BRAND_Toothy Electric Soft Brush,BRAND_Toothy Fresh,BRAND_Toothy Hard Brush,BRAND_Toothy Kids,BRAND_Toothy Salt,BRAND_Toothy Sensitive,BRAND_Turquoise,BRAND_Umber,BRAND_Venus,BRAND_Violet
0,202408,2610.18,16,0.004560,2690.909091,202208,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
1,202408,245.45,2,0.000500,245.454545,202208,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,202408,277.14,32,0.000768,285.714286,202208,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,202408,345.45,2,0.000750,345.454545,202208,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,202408,138.57,16,0.000400,142.857143,202208,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
data=pd.concat([data,status],axis=1)

In [ ]:
data.head()

In [ ]:
data.drop(['MNTH_CODE', 'TRANS_DATE', 'START_DATE', 'SALES_VALUE', 'SALES_UNITS',
       'SALES_VOLUME', 'SALES_PTR_VALUE', 'OC_CODE', 'DISTRIBUTOR_CODE',
       'OUTLET_CODE', 'CITY', 'STATE', 'COUNTY', 'STREET', 'PRODUCT_CODE',
       'CATEGORY', 'SUBCATEGORY', 'BRAND'],axis=1,inplace=True)

In [ ]:
data.head()

In [ ]:
X=data.drop(['CUSTOMER_ID','TRANS_DATE','START_DATE'],axis=1)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
Scaler=MinMaxScaler()

In [ ]:
X.head()

In [ ]:
Scaler.fit(X)
X_Train_scaled=Scaler.transform(X)

In [ ]:
X_Train_scaled

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

# Assuming X_Train_scaled is already defined and scaled

# Initialize PCA with the number of components you want
pca = PCA(n_components=2)

# Fit and transform the scaled training data
X_Train_pca = pca.fit_transform(X_Train_scaled)

# Convert the PCA result to a DataFrame
X_Train_pca_df = pd.DataFrame(X_Train_pca, columns=['PC1', 'PC2'])

# Perform K-means clustering
kmeans = KMeans(n_clusters=4)  # Adjust the number of clusters as needed
kmeans.fit(X_Train_pca_df)

# Add the cluster labels to the DataFrame
X_Train_pca_df['Cluster'] = kmeans.labels_

# Plot the clusters
plt.figure(figsize=(10, 7))
plt.scatter(X_Train_pca_df['PC1'], X_Train_pca_df['PC2'], c=X_Train_pca_df['Cluster'], cmap='viridis')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('PCA and K-means Clustering')
plt.show()

In [ ]:
from sklearn.cluster import KMeans  
 
wcss_list= []  #Initializing the list for the values of WCSS  
 
#Using for loop for iterations from 1 to 10.  
 
for i in range(1, 11):  
 
    kmeans = KMeans(n_clusters=i, init='k-means++', random_state= 42)  
 
    kmeans.fit(X_Train_pca_df)  
 
    wcss_list.append(kmeans.inertia_)  
 
plt.plot(range(1, 11), wcss_list)  
 
plt.title('The Elobw Method Graph')  
 
plt.xlabel('Number of clusters(k)')  
 
plt.ylabel('wcss_list')  
 
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

# Assuming X_Train_scaled is already defined and scaled

# Initialize PCA with the number of components you want
pca = PCA(n_components=2)

# Fit and transform the scaled training data
X_Train_pca = pca.fit_transform(X_Train_scaled)

# Convert the PCA result to a DataFrame
X_Train_pca_df = pd.DataFrame(X_Train_pca, columns=['PC1', 'PC2'])

# Perform K-means clustering
kmeans = KMeans(n_clusters=4)  # Adjust the number of clusters as needed
kmeans.fit(X_Train_pca_df)

# Add the cluster labels to the DataFrame
X_Train_pca_df['Cluster'] = kmeans.labels_

# Plot the clusters
plt.figure(figsize=(10, 7))
plt.scatter(X_Train_pca_df['PC1'], X_Train_pca_df['PC2'], c=X_Train_pca_df['Cluster'], cmap='viridis')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('PCA and K-means Clustering')
plt.show()

In [ ]:
combined_df = pd.concat([df, X_Train_pca_df], axis=1)

In [ ]:
combined_df.head()

In [ ]:
combined_df['Cluster'].value_counts()

In [ ]:
pip install snowflake-connector-python

In [ ]:
import pandas as pd
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas

In [ ]:
combined_df['Cluster'].value_counts()


In [ ]:
combined_df['ClusterID'] = combined_df['Cluster']
 
di = {0: "CLUSTER_0", 1: "CLUSTER_1",2: "CLUSTER_2", 3: "CLUSTER_3"}
combined_df.replace({"ClusterID": di})

In [ ]:
#Code to write back dataframe to Snowflake
 
ins_train_sf=my_session.createDataFrame(
        combined_df.values.tolist(),
        schema=combined_df.columns.tolist())
ins_train_sf.write.mode("overwrite").save_as_table("USECASE_DB_BFSI.BRONZE.BFSI_FINAL")